In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv("data/train_combine.csv")
train.head()

,id,author,geometry,pressure [MPa],mass_flux [kg/m2-s],x_e_out [-],D_e [mm],D_h [mm],length [mm],chf_exp [MW/m2]
0,0,Thompson,tube,7.00,3770.0,0.1754,NaN,10.8,432.0,3.6
1,1,Thompson,tube,NaN,6049.0,-0.0416,10.3,10.3,762.0,6.2
2,2,Thompson,NaN,13.79,2034.0,0.0335,7.7,7.7,457.0,2.5
3,3,Beus,annulus,13.79,3679.0,-0.0279,5.6,15.2,2134.0,3.0
4,5,NaN,NaN,17.24,3648.0,-0.0711,NaN,1.9,696.0,3.6


In [3]:
y = train['x_e_out [-]']
X = train.drop(columns=['id','x_e_out [-]'])

In [4]:
X.head()

,author,geometry,pressure [MPa],mass_flux [kg/m2-s],D_e [mm],D_h [mm],length [mm],chf_exp [MW/m2]
0,Thompson,tube,7.00,3770.0,NaN,10.8,432.0,3.6
1,Thompson,tube,NaN,6049.0,10.3,10.3,762.0,6.2
2,Thompson,NaN,13.79,2034.0,7.7,7.7,457.0,2.5
3,Beus,annulus,13.79,3679.0,5.6,15.2,2134.0,3.0
4,NaN,NaN,17.24,3648.0,NaN,1.9,696.0,3.6


In [5]:
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import make_pipeline

In [6]:
# set up preprocessing for numeric columns
imp_knn = KNNImputer(n_neighbors=20)
scaler = StandardScaler()

In [7]:
# set up preprocessing for categorical columns
from sklearn.impute import SimpleImputer
imp_constant = SimpleImputer(strategy='constant')
ohe = OneHotEncoder(handle_unknown='ignore')

In [8]:
from sklearn.compose import make_column_selector, make_column_transformer

In [9]:
# select columns by data type
num_cols = make_column_selector(dtype_include='number')
cat_cols = make_column_selector(dtype_exclude='number')

In [10]:
preprocessor = make_column_transformer(
    (make_pipeline(imp_knn, scaler), num_cols),
    (make_pipeline(imp_constant, ohe), cat_cols))

In [11]:
import lightgbm as lgb
lgbm = lgb.LGBMRegressor(n_estimators=125,random_state = 318) #0.0054248

from catboost import CatBoostRegressor
cb = CatBoostRegressor(n_estimators=1000,random_state = 318,verbose=0, objective='RMSE') #0.005509

from sklearn.ensemble import GradientBoostingRegressor,HistGradientBoostingRegressor
hgbr = HistGradientBoostingRegressor(max_iter=1000,random_state=318)
gbr = GradientBoostingRegressor(n_estimators=750)

import xgboost as xgb
xgb = xgb.XGBRegressor(n_estimators=35,random_state = 318) #0.005579

from sklearn.ensemble import VotingRegressor
vr1 = VotingRegressor([('hgbr',hgbr),('lgbm',lgbm),('cb',cb)],weights=(1,2,1))

In [12]:
pipe_vr1 = make_pipeline(preprocessor,vr1)

In [13]:
pipe_vr1.steps

[('columntransformer',
  ColumnTransformer(transformers=[('pipeline-1',
                                   Pipeline(steps=[('knnimputer',
                                                    KNNImputer(n_neighbors=20)),
                                                   ('standardscaler',
                                                    StandardScaler())]),
                                   <sklearn.compose._column_transformer.make_column_selector object at 0x00000193CE704BB0>),
                                  ('pipeline-2',
                                   Pipeline(steps=[('simpleimputer',
                                                    SimpleImputer(strategy='constant')),
                                                   ('onehotencoder',
                                                    OneHotEncoder(handle_unknown='ignore'))]),
                                   <sklearn.compose._column_transformer.make_column_selector object at 0x00000193C92598E0>)])),
 ('votingregre

In [23]:
from sklearn.model_selection import GridSearchCV

params1 = {}
params1['columntransformer__pipeline-1__knnimputer__n_neighbors'] = [53,54,55,56,57]

grid = GridSearchCV(pipe_vr1, params1, scoring='neg_mean_squared_error')
%time grid.fit(X,y) #55：0.0053976

Wall time: 7min 27s


GridSearchCV(estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('pipeline-1',
                                                                         Pipeline(steps=[('knnimputer',
                                                                                          KNNImputer(n_neighbors=20)),
                                                                                         ('standardscaler',
                                                                                          StandardScaler())]),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x00000193CE704BB0>),
                                                                        ('pipeline-2',
                                                                         Pipeline(steps=[('simpleimputer',
                                                     

In [24]:
grid.best_params_

{'columntransformer__pipeline-1__knnimputer__n_neighbors': 55}

In [25]:
grid.best_score_

-0.005397611384802032

- best one

In [26]:
#处理test值
test = pd.read_csv('data/test.csv')
X_test = test.drop(columns=['id','x_e_out [-]'])

In [27]:
result = grid.predict(X_test)

In [28]:
result

array([-0.00859985, -0.09280522, -0.0401979 , ...,  0.02870287,
       -0.03745832, -0.01270881])

In [29]:
pd.DataFrame({'id':test.id, 'x_e_out [-]':result}).set_index('id').to_csv('sub_combine_tuningk_knn_impute_vr.csv')

In [ ]:
- 0.75136